## 1. Basic TensorFlow Operations

In [2]:
import tensorflow as tf

# Define constants
a = tf.constant(2)
b = tf.constant(3)

# Perform operations
sum_ab = tf.add(a, b)
prod_ab = tf.multiply(a, b)

#Initialize a session and run the operations
print("Sum:", sum_ab.numpy()) #output: sum: 5
print("Product:", prod_ab.numpy()) #Output: Product: 6

Sum: 5
Product: 6


In [4]:
## 2. Linear Regression Example
import numpy as np

# Data: y = 2x + 1
X = np.array([0, 1, 2, 3, 4], dtype=np.float32)
Y = np.array([1, 3, 5, 7, 9], dtype=np.float32)

#Variables (trainable weights)
W = tf.Variable(0.0)
b = tf.Variable(0.0)

# Linear model
def model(X):
    return W * X + b

# Loss function (Mean squared Error)
def loss_fn(y_true, y_pred):
    return tf.reduce_mean(tf.square(y_true - y_pred))

#optimizer
optimizer = tf.optimizers.SGD(learning_rate=0.01)

#Training
for epoch in range(100):
    with tf.GradientTape() as tape:
        y_pred = model(X)
        loss = loss_fn(Y, y_pred)
    gradients = tape.gradient(loss, [W, b])
    optimizer.apply_gradients(zip(gradients, [W, b]))

#Output the trained weights
print(f"W: {W.numpy()}, b: {b.numpy()}")

W: 2.051225423812866, b: 0.8539638519287109


The script implements a simple linear regression model in TensorFlow. It trains on the data y = 2x + 1 and uses gradient descent for optimization. After 100 epochs, the model should learn parameters W = 2 and b = 1

## 3. Neural Network Example (Multi-Layer Perceptron)

In [5]:
from tensorflow.keras import layers, models

#Sample dataset (MNIST handwritten digits)
mnist = tf.keras.datasets.mnist
(X_train, y_train), (X_test, y_test) = mnist.load_data()

# pre-process data
X_train, X_test = X_train / 255.0, X_test / 255.0

#Build the model
model = models.Sequential([
    layers.Flatten(input_shape = (28, 28)),
    layers.Dense(128, activation = 'relu'),
    layers.Dense(10, activation = 'softmax')
])

#compile the model
model.compile(optimizer = 'adam',
              loss = 'sparse_categorical_crossentropy',
              metrics = ['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs = 5)

#Evaluate the model
test_loss, test_acc = model.evaluate(X_test, y_test)

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 4s 0us/step


C:\Users\kandu\anaconda3\Lib\site-packages\keras\src\layers\reshaping\flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.8780 - loss: 0.4290
Epoch 2/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - accuracy: 0.9655 - loss: 0.1164
Epoch 3/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - accuracy: 0.9766 - loss: 0.0779
Epoch 4/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9825 - loss: 0.0563
Epoch 5/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - accuracy: 0.9867 - loss: 0.0443
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 949us/step - accuracy: 0.9705 - loss: 0.0948


## 4. CNN Fine-Tuning (Last Layer)

In [8]:
from tensorflow.keras import layers, models

#Load pre-trained model (MobileNetV2 without the top layer)
base_model = tf.keras.applications.MobileNetV2(input_shape = (224, 224, 3),
                                               include_top = False,
                                               weights = 'imagenet')

#Freeze the base model layers
base_model.trainable = False

# Add a custom top layer
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(10, activation = 'softmax')
])

#compilr the model
model.compile(optimizer = 'adam',
              loss = 'sparse_categorical_crossentropy',
              metrics = ['accuracy'])

#Dummy data for training (replace with real data)
X_train = tf.random.normal([32, 224, 224, 3]) #32 random images
y_train = tf.random.uniform([32], maxval = 10, dtype = tf.int32)

# Train the top layer
model.fit(X_train, y_train, epochs = 3)

#Fine-tuning the base model (unfreeze some layers)
base_model.trainable = True
model.compile(optimizer = tf.keras.optimizers.Adam(1e-5), #lower learning rate
              loss = 'sparse_categorical_crossentropy',
              metrics = ['accuracy'])

# Fine-tune the whole model
model.fit(X_train, y_train, epochs = 3)

Epoch 1/3
1/1 ━━━━━━━━━━━━━━━━━━━━ 7s 7s/step - accuracy: 0.0312 - loss: 2.9435
Epoch 2/3
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 365ms/step - accuracy: 0.0625 - loss: 2.5729
Epoch 3/3
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 357ms/step - accuracy: 0.1562 - loss: 2.3409
Epoch 1/3
1/1 ━━━━━━━━━━━━━━━━━━━━ 40s 40s/step - accuracy: 0.0625 - loss: 2.4421
Epoch 2/3
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - accuracy: 0.1562 - loss: 2.2602
Epoch 3/3
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - accuracy: 0.2188 - loss: 2.0900
